Make sure to put the train.csv and test.csv in a folder called data in the home directory.

# New Section

Ankith: Neural network

Jenny: XGBoost

Vedu: Random Forest

Ethan: Support Vector Machines

Vidit: k-Nearest Neighbors

In [12]:
%pylab inline
# Pandas is a nice utilitiy that enables some easy data manipulation, especially from a csv
import pandas as pd
# Numpy lets us work with arrays
import numpy as np
import random
# Sklearn provides various modules with a common API
from sklearn import svm, tree, neighbors, neural_network
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score

%pylab is deprecated, use %matplotlib inline and import the required libraries.
Populating the interactive namespace from numpy and matplotlib


C:\Users\ankit\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\IPython\core\magics\pylab.py:162: UserWarning: pylab import has clobbered these variables: ['random']
`%matplotlib` prevents importing * from pylab and numpy
  warn("pylab import has clobbered these variables: %s"  % clobbered +


In [13]:
train_data = pd.read_csv('../data/train.csv')
test_data = pd.read_csv('../data/test.csv')

In [14]:
train_data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [15]:
test_data.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [16]:
print(train_data.columns[train_data.isna().any()].tolist())

['Age', 'Cabin', 'Embarked']


In [17]:
random.seed(100)
np.random.seed(100)

train_data.drop(columns=['Name', 'Ticket', 'Cabin'], inplace=True)
train_data.set_index(keys=['PassengerId'], drop=True, inplace=True)

test_data.drop(columns=['Name', 'Ticket', 'Cabin'], inplace=True)
test_data.set_index(keys=['PassengerId'], drop=True, inplace=True)

train_nan_map = {'Age': train_data['Age'].mean(), 'Fare': train_data['Fare'].mean(), 'Embarked': train_data['Embarked'].mode()[0]}
test_nan_map = {'Age': test_data['Age'].mean(), 'Fare': test_data['Fare'].mean(), 'Embarked': test_data['Embarked'].mode()[0]}

train_data.fillna(value=train_nan_map, inplace=True)
test_data.fillna(value=test_nan_map, inplace=True)

columns_map = {'Embarked': {'C': 0, 'Q': 1, 'S': 2}, 'Sex': {'male': 0, 'female': 1}}
train_data.replace(columns_map, inplace=True)
test_data.replace(columns_map, inplace=True)

X_train = train_data.loc[:, train_data.columns != 'Survived']
y_train = train_data.loc[:, 'Survived']

X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=0.33, random_state=10)

In [18]:
print(X_train.head())
print(y_train.head())
print(test_data.head())
print(f"Test Mean: {y_test.mean()}")
print(f"Train Mean: {y_train.mean()}")

             Pclass  Sex        Age  SibSp  Parch    Fare  Embarked
PassengerId                                                        
464               2    0  48.000000      0      0  13.000         2
160               3    0  29.699118      8      2  69.550         2
48                3    1  29.699118      0      0   7.750         1
403               3    1  21.000000      1      0   9.825         2
619               2    1   4.000000      2      1  39.000         2
PassengerId
464    0
160    0
48     1
403    0
619    1
Name: Survived, dtype: int64
             Pclass  Sex   Age  SibSp  Parch     Fare  Embarked
PassengerId                                                    
892               3    0  34.5      0      0   7.8292         1
893               3    1  47.0      1      0   7.0000         2
894               2    0  62.0      0      0   9.6875         1
895               3    0  27.0      0      0   8.6625         2
896               3    1  22.0      1      1  12.2875 

In [19]:
train_data.columns[train_data.isna().any()].tolist()
train_data[train_data['Embarked'].isna() == True]

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
PassengerId,,,,,,,,


In [20]:
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

# Scale the data using StandardScaler
scaler = StandardScaler()
scaler.fit(X_train)
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Set random seeds
random.seed(12)
np.random.seed(12)

# Initialize the KNeighborsClassifier
knn_cls = KNeighborsClassifier(n_neighbors=5)  # You can adjust the number of neighbors (k) as needed

# Fit the KNN classifier to the scaled training data
knn_cls.fit(X_train_scaled, y_train)

# Predict on the scaled test data
y_pred_knn = knn_cls.predict(X_test_scaled)  # Use X_test_scaled, not test_data.values

# Calculate the accuracy on the test set
accuracy_knn = accuracy_score(y_test, y_pred_knn)
print(f"KNN Accuracy: {accuracy_knn}")

# Confusion matrix
confusion_matrix = confusion_matrix(y_test, y_pred_knn)
print("Confusion Matrix:")
print(confusion_matrix)

# False Negative Rate (FNR) and False Positive Rate (FPR)
FNR = confusion_matrix[1][0] / (confusion_matrix[1][0] + confusion_matrix[1][1])
FPR = confusion_matrix[0][1] / (confusion_matrix[0][0] + confusion_matrix[0][1])
print(f"FNR: {FNR}, FPR: {FPR}")


KNN Accuracy: 0.8135593220338984
Confusion Matrix:
[[166  25]
 [ 30  74]]
FNR: 0.28846153846153844, FPR: 0.13089005235602094


In [21]:
def save_predictions_to_csv(predictions: list, filename: str = "titanic_prediction_group4_vidit.csv") -> None:
    """
    Save the predictions to a CSV file.

    Args:
    - predictions (list): List of predictions to save.
    - filename (str, optional): Name of the CSV file to save to. Defaults to "titanic_prediction_knn.csv".

    Returns:
    None
    """
    df = pd.DataFrame(predictions, columns=["PassengerID"])
    df.to_csv(filename, index=False)

In [22]:
passid_prediction_knn = {'PassengerId': test_data.index, 'Survived': knn_cls.predict(test_data.values)}
df_predictions_knn = pd.DataFrame(passid_prediction_knn)
df_predictions_knn.to_csv("titanic_predictions_group4_vidit.csv", index=False)